In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Carregando os dados
df = pd.read_csv('../input/airline-passenger-satisfaction/train.csv')
test = pd.read_csv('../input/airline-passenger-satisfaction/test.csv')

df.shape, test.shape

In [ ]:
# Juntando os dataframes
df_all = df.append(test)

df_all.shape

In [ ]:
# Verificando o dataframe
df_all.info()

In [ ]:
# Vamos verificar os dados 
df_all.head(10).T

In [ ]:
# Substituindo os espaços por underscore
df_all.columns = [c.replace(' ', '_') for c in df_all.columns]

In [ ]:
#procurando os valores nulos
df_all.isna().sum()

In [ ]:
# Apagando colunas desnecessarias
df_all = df_all.drop('Unnamed:_0', axis=1)
df_all = df_all.drop('id', axis=1)

In [ ]:
# Observando a estatistica descritiva
df_all.describe().round(2)

In [ ]:
# Imputando os valores nulos pela mediana
df_all['Arrival_Delay_in_Minutes'] = df_all['Arrival_Delay_in_Minutes'].fillna(df_all['Arrival_Delay_in_Minutes'].median())
df_all

In [ ]:
#Observando a matriz de correlação
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(20,15))
sns.heatmap(df_all.corr(),annot=True,cmap='coolwarm')
plt.tight_layout

In [ ]:
# Quais colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

In [ ]:
# Olhando a coluna Customer_Type
df_all['Customer_Type'].value_counts()

In [ ]:
# Olhando a coluna Gender
df_all['Gender'].value_counts()

In [ ]:
# Olhando a coluna Type_of_Travel
df_all['Type_of_Travel'].value_counts()

In [ ]:
# Olhando a coluna Class
df_all['Class'].value_counts()

In [ ]:
# Olhando a coluna satisfaction
df_all['satisfaction'].value_counts()

In [ ]:
# Checando o desbalanceamento 

fig = plt.figure(figsize = (8,5))
df_all.satisfaction.value_counts(normalize = True).plot(kind='bar', color= ['darkorange','steelblue'], alpha = 0.9, rot=0)
plt.show()

In [ ]:
# Criando dummys para todas as colunas categoricas, com exceção da target
df_all = pd.get_dummies(df_all, columns=['Gender', 'Customer_Type', 'Type_of_Travel','Class'])
df_all

In [ ]:
# olhando os dados
df_all.head().T

In [ ]:
# Vrificando novamente os tipos dos dados
df_all.info()

In [ ]:
# Treinamento do modelo

# Separando o dataframe

# Importando o train_test_split
from sklearn.model_selection import train_test_split

# Separando treino e teste
train, test = train_test_split(df_all, test_size=0.20, random_state=42)

# Separando treino e validação
train, valid = train_test_split(train, test_size=0.20, random_state=42)

train.shape, valid.shape, test.shape

In [ ]:
# definindo colunas de entrada
feats = [c for c in df_all.columns if c not in ['satisfaction']]

feats

In [ ]:
# treinar o modelo

# Importando o modelo
from sklearn.ensemble import RandomForestClassifier

# Instanciar o modelo
rf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)

In [ ]:
# treinar o modelo
rf.fit(train[feats], train['satisfaction'])

In [ ]:
# Prevendo os dados de validação
preds_val = rf.predict(valid[feats])

preds_val

In [ ]:
# Avaliando o desempenho do modelo

# Importando a metrica
from sklearn.metrics import accuracy_score

In [ ]:
# Acurácia das previsões de validação
accuracy_score(valid['satisfaction'], preds_val)

In [ ]:
# Medindo a acurácia nos dados de teste
preds_test = rf.predict(test[feats])

accuracy_score(test['satisfaction'], preds_test)

In [ ]:
# Olhando a coluna satisfaction do dataframe completo
df_all['satisfaction'].value_counts(normalize=True)

In [ ]:
# Olhando a coluna satisfaction do dataframe completo
df['satisfaction'].value_counts()

In [ ]:
# Avaliando a importancia de cada coluna (cada variável de entrada)
fig=plt.figure(figsize=(10, 15))
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()

In [ ]:
# importando a bilbioteca para plotar o gráfico de Matriz de Confusão
import scikitplot as skplt

In [ ]:
# Matriz de Confusão - Dados de Validação
skplt.metrics.plot_confusion_matrix(valid['satisfaction'], preds_val)

In [ ]:
# Matriz de Confusão - Dados de Teste
skplt.metrics.plot_confusion_matrix(test['satisfaction'], preds_test)